### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


### Web scraping to extract the neighborhoods dataset

In [2]:
dfs=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=dfs[0]

for i in df.Borough:
    df=df.fillna(i)

df=df[df.Borough != 'Not assigned'].reset_index(drop=True)    
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')

df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
# Check the dataset size
df.shape

(103, 3)

### Importing the dataset with geographical coordinates

In [4]:
coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [5]:
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the datasets

In [6]:
neigh=pd.merge(df, coord)

neigh.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Exploring venues in each neighborhood using Foursquare API

In [7]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'OA35LAYGTJPGETIF1QWUA0C5F3EKWOVY113415BLNE5ZLUKN' # your Foursquare ID
CLIENT_SECRET = 'QPOZUBTPRF0E2N24V05D3IOU1BS3RXIMGZAGILJVB0WERHR1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OA35LAYGTJPGETIF1QWUA0C5F3EKWOVY113415BLNE5ZLUKN
CLIENT_SECRET:QPOZUBTPRF0E2N24V05D3IOU1BS3RXIMGZAGILJVB0WERHR1


In [8]:
# Obtain 100 venues in each neighborhood within a radius of 1000 meters

LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# Create a new dataframe with extracted venues
toronto_venues = getNearbyVenues(names=neigh['Neighborhood'],
                                   latitudes=neigh['Latitude'],
                                   longitudes=neigh['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [10]:
# Display toronto_venues data
print(toronto_venues.shape)
toronto_venues.head()

(4893, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [11]:
# Get the number of venues in neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,45,45,45,45,45,45
"Alderwood, Long Branch",26,26,26,26,26,26
"Bathurst Manor, Wilson Heights, Downsview North",30,30,30,30,30,30
Bayview Village,15,15,15,15,15,15
"Bedford Park, Lawrence Manor East",43,43,43,43,43,43
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",12,12,12,12,12,12
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",47,47,47,47,47,47


In [12]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 331 uniques categories.


### One hot encoding

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# getting the column index of neighborhood in toronto_onehot
toronto_onehot.columns.get_loc("Neighborhood")

216

In [14]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[216]] + list(toronto_onehot.columns[:216]) + list(toronto_onehot.columns[217:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,

In [15]:
toronto_onehot.shape

(4893, 331)

In [16]:
# Group by neighborhoods and display average frequency
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,

### Display top 10 venue for each neighbourhood

In [17]:
# Create function to sort venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
# Create a new dataframe displaying top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Caribbean Restaurant,Sandwich Place,Bakery,Bubble Tea Shop,Bank,Restaurant,Sushi Restaurant
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Convenience Store,Pharmacy,Park,Moroccan Restaurant,Athletics & Sports,Gas Station,Liquor Store,Sandwich Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Coffee Shop,Supermarket,Gas Station,Mobile Phone Shop,Dog Run,Sushi Restaurant,Sandwich Place,Fried Chicken Joint
3,Bayview Village,Bank,Grocery Store,Gas Station,Japanese Restaurant,Café,Chinese Restaurant,Park,Trail,Intersection,Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Park,Bank,Restaurant,Baby Store,Bagel Shop,Bakery,Butcher


### Cluster neighborhoods

In [19]:
# define number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means model
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 3, 4, 4, 4, 0, 4])

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Caribbean Restaurant,Sandwich Place,Bakery,Bubble Tea Shop,Bank,Restaurant,Sushi Restaurant
1,0,"Alderwood, Long Branch",Discount Store,Pizza Place,Convenience Store,Pharmacy,Park,Moroccan Restaurant,Athletics & Sports,Gas Station,Liquor Store,Sandwich Place
2,0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Coffee Shop,Supermarket,Gas Station,Mobile Phone Shop,Dog Run,Sushi Restaurant,Sandwich Place,Fried Chicken Joint
3,3,Bayview Village,Bank,Grocery Store,Gas Station,Japanese Restaurant,Café,Chinese Restaurant,Park,Trail,Intersection,Restaurant
4,3,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Park,Bank,Restaurant,Baby Store,Bagel Shop,Bakery,Butcher


In [21]:
# merge neigh with neighborhoods_venues_sorted to add geographical coordinates for each neighborhood

toronto_merged = neigh.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Pharmacy,Shopping Mall,Bus Stop,Convenience Store,Pizza Place,Food & Drink Shop,Skating Rink,Fish & Chips Shop,Road
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Coffee Shop,Men's Store,Lounge,Golf Course,Park,Grocery Store,Café,Gym / Fitness Center,Portuguese Restaurant,Sporting Goods Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Café,Pub,Park,Theater,Diner,Breakfast Spot,Restaurant,Italian Restaurant,Bakery
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Sushi Restaurant,Clothing Store,Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Dessert Shop,Accessories Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Coffee Shop,Sushi Restaurant,Park,Ramen Restaurant,Burger Joint,Café,Japanese Restaurant,Dance Studio,Bubble Tea Shop,Restaurant


In [22]:
# Check missing values in toronto_merged
toronto_merged.isnull().sum()

Postal Code               0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            1
1st Most Common Venue     1
2nd Most Common Venue     1
3rd Most Common Venue     1
4th Most Common Venue     1
5th Most Common Venue     1
6th Most Common Venue     1
7th Most Common Venue     1
8th Most Common Venue     1
9th Most Common Venue     1
10th Most Common Venue    1
dtype: int64

In [23]:
# Drop missing value and convert cluster labels from float to integer
toronto_merged.dropna(axis=0, inplace=True)

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

### Visualize clusters on the map

In [25]:
#Obtaining the geographical coordinates of the City of Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [26]:
from IPython.display import HTML, display

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyze clusters

In [27]:
# Display the number of neighborhoods for each cluster
value_counts = toronto_merged['Cluster Labels'].value_counts()
cluster_count = pd.DataFrame(value_counts).reset_index()
cluster_count.columns = ['Clusters', 'Number of Neighborhoods']
cluster_count

,Clusters,Number of Neighborhoods
0,0,37
1,4,36
2,3,27
3,2,1
4,1,1


### Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,0,Park,Pharmacy,Shopping Mall,Bus Stop,Convenience Store,Pizza Place,Food & Drink Shop,Skating Rink,Fish & Chips Shop,Road
5,"Islington Avenue, Humber Valley Village",0,Pharmacy,Skating Rink,Bakery,Golf Course,Shopping Mall,Grocery Store,Café,Park,Bank,Playground
8,"Parkview Hill, Woodbine Gardens",0,Pizza Place,Brewery,Gym / Fitness Center,Fast Food Restaurant,Soccer Stadium,Intersection,Gastropub,Bank,Bakery,Rock Climbing Spot
10,Glencairn,0,Grocery Store,Fast Food Restaurant,Pizza Place,Italian Restaurant,Gas Station,Coffee Shop,Bank,Sushi Restaurant,Latin American Restaurant,Fish Market
11,"West Deane Park, Princess Gardens, Martin Grov...",0,Park,Hotel,Pizza Place,Convenience Store,Restaurant,Grocery Store,Gym,Clothing Store,Bank,Fish & Chips Shop
14,Woodbine Heights,0,Coffee Shop,Park,Pizza Place,Thai Restaurant,Athletics & Sports,Sandwich Place,Pharmacy,Farmers Market,Beer Store,Liquor Store
16,Humewood-Cedarvale,0,Pizza Place,Convenience Store,Coffee Shop,Park,Israeli Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Field,Frozen Yogurt Shop,Gastropub
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,Coffee Shop,Intersection,Liquor Store,Fish & Chips Shop,Beer Store,Shopping Mall,Shopping Plaza,Grocery Store,Café,Skating Rink
18,"Guildwood, Morningside, West Hill",0,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Fried Chicken Joint,Liquor Store,Discount Store,Supermarket,Beer Store,Electronics Store
21,Caledonia-Fairbanks,0,Bus Stop,Pharmacy,Park,Cosmetics Shop,Bank,Bakery,Beer Store,Discount Store,Falafel Restaurant,Grocery Store


### Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"York Mills, Silver Hills",1,Park,Pool,Zoo,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


### Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Northwest, West Humber - Clairville",2,Coffee Shop,Lounge,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market


### Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Lawrence Manor, Lawrence Heights",3,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Sushi Restaurant,Clothing Store,Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Dessert Shop,Accessories Store
6,"Malvern, Rouge",3,Coffee Shop,Restaurant,Fast Food Restaurant,Trail,Spa,Bank,Bakery,Chinese Restaurant,Caribbean Restaurant,Auto Workshop
7,Don Mills,3,Coffee Shop,Restaurant,Japanese Restaurant,Supermarket,Bank,Pizza Place,Asian Restaurant,Gym,Burger Joint,Beer Store
12,"Rouge Hill, Port Union, Highland Creek",3,Breakfast Spot,Italian Restaurant,Burger Joint,Playground,Park,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
13,Don Mills,3,Coffee Shop,Restaurant,Japanese Restaurant,Supermarket,Bank,Pizza Place,Asian Restaurant,Gym,Burger Joint,Beer Store
19,The Beaches,3,Pub,Pizza Place,Coffee Shop,Breakfast Spot,Japanese Restaurant,Beach,Park,Health Food Store,Bakery,Burger Joint
23,Leaside,3,Coffee Shop,Grocery Store,Furniture / Home Store,Electronics Store,Sporting Goods Shop,Burger Joint,Brewery,Restaurant,Sandwich Place,Department Store
26,Cedarbrae,3,Bakery,Coffee Shop,Gas Station,Indian Restaurant,Bank,Grocery Store,Fast Food Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint
29,Thorncliffe Park,3,Coffee Shop,Grocery Store,Indian Restaurant,Afghan Restaurant,Shopping Mall,Bank,Sandwich Place,Pizza Place,Burger Joint,Supermarket
32,Scarborough Village,3,Ice Cream Shop,Fast Food Restaurant,Bowling Alley,Train Station,Grocery Store,Restaurant,Coffee Shop,Japanese Restaurant,Sandwich Place,Pizza Place


### Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,4,Coffee Shop,Men's Store,Lounge,Golf Course,Park,Grocery Store,Café,Gym / Fitness Center,Portuguese Restaurant,Sporting Goods Shop
2,"Regent Park, Harbourfront",4,Coffee Shop,Café,Pub,Park,Theater,Diner,Breakfast Spot,Restaurant,Italian Restaurant,Bakery
4,"Queen's Park, Ontario Provincial Government",4,Coffee Shop,Sushi Restaurant,Park,Ramen Restaurant,Burger Joint,Café,Japanese Restaurant,Dance Studio,Bubble Tea Shop,Restaurant
9,"Garden District, Ryerson",4,Coffee Shop,Japanese Restaurant,Gastropub,Theater,Hotel,Café,Burrito Place,Italian Restaurant,Seafood Restaurant,Restaurant
15,St. James Town,4,Coffee Shop,Café,Restaurant,Gastropub,Italian Restaurant,Theater,Bookstore,Hotel,Tea Room,Japanese Restaurant
20,Berczy Park,4,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Grocery Store,Liquor Store,Creperie,Bakery
24,Central Bay Street,4,Coffee Shop,Café,Japanese Restaurant,Park,Clothing Store,Art Gallery,Sushi Restaurant,Burger Joint,Arts & Crafts Store,Juice Bar
25,Christie,4,Korean Restaurant,Café,Coffee Shop,Grocery Store,Mexican Restaurant,Cocktail Bar,Ice Cream Shop,Ethiopian Restaurant,Indian Restaurant,Bar
30,"Richmond, Adelaide, King",4,Coffee Shop,Café,Hotel,Theater,Japanese Restaurant,Tea Room,Pizza Place,Furniture / Home Store,Concert Hall,Plaza
31,"Dufferin, Dovercourt Village",4,Café,Coffee Shop,Park,Bakery,Sushi Restaurant,Grocery Store,Bar,Portuguese Restaurant,Restaurant,Italian Restaurant
